In [2]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Project Path Setup
from setup_project_path import check_and_add_project_path  # Script to check and add project path
check_and_add_project_path()

# Import Custom Modules
from src.all_custom_modules import DiscrError, DlrKernel, cf, DataInterp, plot_data, Hdf5Kernel # Consolidated custom modules import


In [3]:
#_____Set Parameters______

params = {
    "m": None,  # to be updated for each computation
    "n": None,  # to be updated for each computation
    "N_max": None,  # not specified as we will directly pass the time grids "times"
    "delta_t": 0.1,  #this step that we will use below to set the time grids
    "beta": np.inf,  # inverse temperature
    "upper_cutoff": np.inf,  # frequency cutoff for continous integrations
    "h": None,  # to be updated for each computation
    "phi": np.pi/4,#rotation angle in complex frequency plane
}  

#Array specifying all values for the discreitzation parameter, h, that should be evaluated
h_vals = np.log( np.arange(2., 1.0, -0.125))  
nbr_hs = len(h_vals)

#Array specifying all values for the total number of time steps, N_max, that should be evaluated
N_maxs = np.concatenate((np.arange(20,1000,500),np.arange(1100,1000,1000)))
nbr_N_maxs = len(N_maxs)

#Array specifying all values for inverse temperature, beta, that should be evaluated
betas = [0,20, 100, 10000]
nbr_betas = len(betas)

#grid dimensions for grid spanned by the following array: N_maxs, h_vals and betas.
param_grid_dims = (nbr_hs, nbr_N_maxs, nbr_betas)

In [4]:
#Define filename of hdf5 file holding the data and create Hdf5Kernel object associated with the file
filename = f'../data/delta_t={params["delta_t"]}.h5'

#Create instance of Hdf5Kernel
h5_kernel = Hdf5Kernel(filename=filename)

In [5]:
#for each point on the parameter grid, compute the discretization error and the corresponding DlrKernel object, 
#and store in hdf5 file associated with 'h5_kernel'.
#create hdf5 file to write to
h5_kernel.create_file(kernel_dims= param_grid_dims)

for b, beta in enumerate(betas):
    for tau, N_max in enumerate(N_maxs):

        cf.update_parameters(params, {"N_max": N_max})
        cf.update_parameters(params, {"beta": beta})

        #compute continous-frequency integral
        times = cf.set_time_grid(N_max=params["N_max"], delta_t= params["delta_t"])
        cont_integral = np.array([cf.cont_integral(t, params["beta"], params["upper_cutoff"])for t in times])  
        
        for h, h_val in enumerate(h_vals):
            
            cf.update_parameters(params, {"h": h_val})

            # Create DiscrError object which holds the error w.r.t. to the continous results, and all associated parameters.
            discr_error = DiscrError(**params, cont_integral_init=cont_integral)  
            discr_error.optimize()  # optimize values for m and n

            #Exit the loop it error is below machine precision, otherwise, append to hdf5 file
            if discr_error.eps < 1.e-14:
                break
            else:
                #create DlrKernel object based on DiscrError object
                kernel = DlrKernel(discr_error)
                #store to hdf5 file
                h5_kernel.append_kernel_element(kernel, (h, tau, b))


/Users/julianthoenniss/Documents/PhD/code/channel_IF/RealTimeID/src/utils/common_funcs.py:234: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  right_segment_cont_real, _ = integrate.quad(
